In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import re
import seaborn as sns
import matplotlib.pyplot as plt


data_folder = 'data'

testdata = pd.read_csv(data_folder + '/ais_test.csv')
traindata = pd.read_csv(data_folder + '/ais_train.csv')

print(testdata.head())
print(traindata.head())


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def preprocess_data(data):
    data = data.drop(['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading', 'VesselName', 'IMO', 'CallSign', 'VesselType', 'Status', 'Length', 'Width', 'Draft', 'Cargo'], axis=1)
    data = data.dropna()
    data = data.drop_duplicates()
    data = data.reset_index(drop=True)
    return data

testdata = preprocess_data(testdata)
traindata = preprocess_data(traindata)

def encode_data(data):
    data = pd.get_dummies(data, columns=['VesselClass'])
    return data

testdata = encode_data(testdata)
traindata = encode_data(traindata)

